# Make SQL Table For Journey Pattern ID's , Distance & Stop ID's

In [56]:
import pandas as pd
import numpy as np

In [40]:
#filter data to those buses that are at the stop. We want to construct a dataframe with a full example of journey pattern id
df = pd.read_hdf("cleaned_store.h5", key="table_name", where="At_Stop == 1")

In [42]:
# Organise the Data
df.sort_values(['Time_Frame', 'Vehicle_ID', 'Vehicle_Journey_ID', 'Timestamp'], ascending=True, inplace=True)
grouped = df.groupby(["Time_Frame", "Vehicle_ID", "Vehicle_Journey_ID"])

In [43]:
def func(group):
    # filter all group whose first row for time is not 0.0, this is just a precaution to get the longest and most correct group
    return group.iloc[0,14] == 0.0


In [44]:
df = grouped.filter(lambda x: func(x))

In [45]:
# Dataframe to hold information
sql_table = pd.DataFrame()

# Get Unique Journey Pattern ID's 
j_patterns = df["Journey_Pattern_ID"].value_counts().index

# Write Loop to add longest example of each JourneyPatternId to a new dataframe
for journey in j_patterns:
    add_df = df[df.Journey_Pattern_ID == journey]
   
    add_df = add_df.groupby(["Time_Frame", "Vehicle_ID", "Vehicle_Journey_ID"])
   
    add_df = sorted(add_df, key=lambda x: len(x[1]), reverse=True)
   
    add_df = add_df[0][1]
           
    # Combine DataFrames
    sql_table = sql_table.append(add_df, ignore_index=True)

In [48]:
sql_table = sql_table[["Journey_Pattern_ID", "Stop_ID", "Distance"]]

In [54]:
# Organise the Data
sql_table.sort_values(["Journey_Pattern_ID", "Distance"], ascending=True, inplace=True)
sql_table.reset_index(inplace=True, drop=True)

C:\Users\Daniele\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
sql_table.to_csv("JourneyPatternID_StopID.csv",index=False)